<a href="https://colab.research.google.com/github/HaniyeDousti/Machine-Learning-Project/blob/main/CaschoolwithLasso.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pydataset

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 55.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pydataset: filename=pydataset-0.2.0-py3-none-any.whl size=15939432 sha256=c7e97129d6c816eeb6352f19448a742ffdf0ffe5700d624e2eb64af514622440
  Stored in directory: /root/.cache/pip/wheels/d7/e5/36/85d319586b4a405d001029d489102f526ce5546248c295932a
Successfully built pydataset


In [11]:
from pydataset import data
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso

**Data Preparation**

In [ ]:
df = pd.DataFrame(data('Caschool'))

In [ ]:
df.head()

,distcod,county,district,grspan,enrltot,teachers,calwpct,mealpct,computer,testscr,compstu,expnstu,str,avginc,elpct,readscr,mathscr
1,75119,Alameda,Sunol Glen Unified,KK-08,195,10.900000,0.510200,2.040800,67,690.799988,0.343590,6384.911133,17.889910,22.690001,0.000000,691.599976,690.000000
2,61499,Butte,Manzanita Elementary,KK-08,240,11.150000,15.416700,47.916698,101,661.200012,0.420833,5099.380859,21.524664,9.824000,4.583333,660.500000,661.900024
3,61549,Butte,Thermalito Union Elementary,KK-08,1550,82.900002,55.032299,76.322601,169,643.599976,0.109032,5501.954590,18.697226,8.978000,30.000002,636.299988,650.900024
4,61457,Butte,Golden Feather Union Elementary,KK-08,243,14.000000,36.475399,77.049202,85,647.700012,0.349794,7101.831055,17.357143,8.978000,0.000000,651.900024,643.500000
5,61523,Butte,Palermo Union Elementary,KK-08,1335,71.500000,33.108601,78.427002,171,640.849976,0.128090,5235.987793,18.671329,9.080333,13.857677,641.799988,639.900024


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 420 entries, 1 to 420
Data columns (total 17 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   distcod   420 non-null    int64  
 1   county    420 non-null    object 
 2   district  420 non-null    object 
 3   grspan    420 non-null    object 
 4   enrltot   420 non-null    int64  
 5   teachers  420 non-null    float64
 6   calwpct   420 non-null    float64
 7   mealpct   420 non-null    float64
 8   computer  420 non-null    int64  
 9   testscr   420 non-null    float64
 10  compstu   420 non-null    float64
 11  expnstu   420 non-null    float64
 12  str       420 non-null    float64
 13  avginc    420 non-null    float64
 14  elpct     420 non-null    float64
 15  readscr   420 non-null    float64
 16  mathscr   420 non-null    float64
dtypes: float64(11), int64(3), object(3)
memory usage: 59.1+ KB


In [ ]:
df.isnull().sum()

distcod     0
county      0
district    0
grspan      0
enrltot     0
teachers    0
calwpct     0
mealpct     0
computer    0
testscr     0
compstu     0
expnstu     0
str         0
avginc      0
elpct       0
readscr     0
mathscr     0
dtype: int64

**Seperating the features and target**

In [ ]:
X = df[['teachers', 'calwpct', 'mealpct', 'compstu', 'expnstu', 'str', 'avginc', 'elpct']]
y = df['testscr']

**Training model**

In [ ]:
regression = LinearRegression()
regression.fit(X, y)
LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)
first_model = (mean_squared_error(y_true=y, y_pred=regression.predict(X)))
first_model

69.0738053013742

**Coefficient for the baseline regression model**

In [ ]:
coef_dict_baseline = {}
for coef, feat in zip(regression.coef_, X.columns):
  coef_dict_baseline[feat] = coef
coef_dict_baseline

{'teachers': 0.00010011947964880743,
 'calwpct': -0.07813766458116585,
 'mealpct': -0.3754719080127329,
 'compstu': 11.914006268826547,
 'expnstu': 0.0015256307099651145,
 'str': -0.19234209691788787,
 'avginc': 0.6211690806021225,
 'elpct': -0.19857026121348387}

**Lasso Model**

In [ ]:
lasso = Lasso(normalize=True)
search = GridSearchCV(estimator=lasso, param_grid={'alpha':np.logspace(-5,2,8)},
                      scoring = 'neg_mean_squared_error', n_jobs=1, refit=True, cv=10)

search.fit(X, y)

In [14]:
print(search.best_params_)
print(abs(search.best_score_))

{'alpha': 1e-05}
85.38831122903972


In [19]:
lasso = Lasso(normalize=True, alpha=1e-05)
lasso.fit(X, y)
second_model = (mean_squared_error(y_true=y, y_pred=lasso.predict(X)))
print(second_model)

69.07380555276043


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_base.py:141: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), Lasso())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)

Set parameter alpha to: original_alpha * np.sqrt(n_samples). 
  warnings.warn(


In [20]:
coef_dict_baseline = {}
for coef, feat in zip(lasso.coef_, X.columns):
  coef_dict_baseline[feat] = coef
coef_dict_baseline

{'teachers': 9.795933425677921e-05,
 'calwpct': -0.07810938255735601,
 'mealpct': -0.37548182158171683,
 'compstu': 11.912164626066946,
 'expnstu': 0.0015254399842507121,
 'str': -0.19225486069458322,
 'avginc': 0.6211695477945165,
 'elpct': -0.1985510490295492}